# 📊 Nelson-Siegel Yield Curve Interactive Analysis

**A Comprehensive Guide to Yield Curve Modeling with Economic Insights**

---

This notebook provides an interactive exploration of the Nelson-Siegel yield curve model with detailed economic and financial explanations. You'll learn how to:

- 🎯 Understand the economic meaning behind yield curve shapes
- 📈 Explore Nelson-Siegel parameters interactively
- 🏛️ Compare Treasury and TIPS yield curves
- 📊 Analyze historical factor evolution
- 💼 Apply insights to real-world bond investment decisions

---

## 📚 Prerequisites

Make sure you have the required packages installed:

```bash
pip install numpy pandas scipy matplotlib ipywidgets
pip install -e .  # Install the nelson_siegel package
```


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
from pathlib import Path

# Add src directory to path (if running from examples directory)
src_path = Path.cwd().parent / "src"
if src_path.exists():
    sys.path.insert(0, str(src_path))

# Import nelson_siegel components
from nelson_siegel import (
    NelsonSiegelModel,
    TreasuryNelsonSiegelModel,
    TIPSNelsonSiegelModel,
    YieldCurveAnalyzer,
    YieldCurvePlotter,
    DataManager
)

# Try to import interactive components
try:
    from nelson_siegel.interactive import (
        InteractiveYieldCurveExplorer,
        HistoricalFactorExplorer,
        create_yield_curve_tutorial
    )
    import ipywidgets as widgets
    from IPython.display import display, HTML
    HAS_WIDGETS = True
    print("✅ Interactive widgets available!")
except ImportError as e:
    HAS_WIDGETS = False
    print(f"⚠️ Interactive widgets not available: {e}")
    print("Install with: pip install ipywidgets jupyter")

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')

print("📦 Libraries loaded successfully!")


## 🎓 Chapter 1: Understanding Yield Curves

### What is a Yield Curve?

A **yield curve** is a graph that shows the relationship between interest rates (yields) and the time to maturity for debt securities of the same credit quality. It's one of the most powerful tools in finance and economics for understanding:

- 📈 **Economic expectations** (growth, inflation, recession risks)
- 🏦 **Monetary policy** stance and future direction
- 💰 **Investment opportunities** and relative value
- ⚡ **Market sentiment** and risk appetite

### Key Economic Concepts

#### 1. **Normal Yield Curve** (Upward Sloping)
- Long-term rates > Short-term rates
- Indicates healthy economic growth expectations
- Reflects compensation for inflation and duration risk

#### 2. **Inverted Yield Curve** (Downward Sloping)
- Short-term rates > Long-term rates
- Historically reliable recession predictor
- Suggests expectations of economic slowdown and falling rates

#### 3. **Flat Yield Curve**
- Similar rates across all maturities
- Often occurs during economic transitions
- May indicate uncertainty about future economic direction


In [ ]:
# Display the educational tutorial if widgets are available
if HAS_WIDGETS:
    tutorial = create_yield_curve_tutorial()
    display(tutorial)
else:
    print("📖 Tutorial: Interactive widgets not available. See markdown content above for yield curve concepts.")


## 🔬 Chapter 2: The Nelson-Siegel Model

### Mathematical Foundation

The Nelson-Siegel model represents yield curves using just **four interpretable parameters**:

$$y(\\tau) = \\beta_0 + \\beta_1 \\left(\\frac{1 - e^{-\\tau/\\lambda}}{\\tau/\\lambda}\\right) + \\beta_2 \\left(\\frac{1 - e^{-\\tau/\\lambda}}{\\tau/\\lambda} - e^{-\\tau/\\lambda}\\right)$$

Where:
- $\\tau$ = time to maturity
- $\\beta_0$ = **Level** (long-term yield)
- $\\beta_1$ = **Slope** (short-term factor)
- $\\beta_2$ = **Curvature** (medium-term factor)
- $\\lambda$ = **Tau** (decay parameter)

### Economic Interpretation of Parameters

#### 🔹 **Level ($\\beta_0$)** - The Long-Term Anchor
- Represents the **long-term yield level**
- Economic meaning: Long-term inflation expectations + real growth + risk premium
- Affects all maturities equally (parallel shifts)
- Typical range: 2-6% for Treasuries, -1% to +3% for TIPS

#### 🔹 **Slope ($\\beta_1$)** - The Short-Term Driver
- Controls the **short-term yield component**
- Economic meaning: Monetary policy expectations and business cycle position
- Positive → Upward sloping curve (normal)
- Negative → Downward sloping curve (inverted)
- Most volatile parameter, responds to Fed policy changes

#### 🔹 **Curvature ($\\beta_2$)** - The Belly Factor
- Affects **medium-term yields** ("belly" of the curve)
- Economic meaning: Supply/demand imbalances, preferred habitat effects
- Positive → Humped curve (medium rates higher)
- Negative → U-shaped curve (medium rates lower)
- Captures local market distortions

#### 🔹 **Tau ($\\lambda$)** - The Decay Speed
- Controls where the **curvature effect peaks**
- Lower tau → curvature affects shorter maturities
- Higher tau → curvature affects longer maturities
- Typical range: 1-5 years


In [ ]:
# Demonstrate the model components with a simple example
def demonstrate_nelson_siegel_components():
    """Show how different parameters affect the yield curve shape."""
    
    # Create maturity range
    maturities = np.linspace(0.25, 30, 100)
    
    # Example parameter sets showing different curve shapes
    examples = {
        'Normal Curve': (0.04, -0.01, 0.005, 2.0),
        'Inverted Curve': (0.035, 0.015, -0.005, 2.0),
        'Humped Curve': (0.04, -0.01, 0.02, 2.0),
        'Flat Curve': (0.035, 0.001, 0.0, 2.0)
    }
    
    plt.figure(figsize=(15, 10))
    
    for i, (name, params) in enumerate(examples.items()):
        plt.subplot(2, 2, i+1)
        
        # Calculate yield curve
        yields = NelsonSiegelModel.model_function(maturities, *params)
        
        plt.plot(maturities, yields * 100, 'b-', linewidth=3, alpha=0.8)
        plt.title(f'{name}\\nβ₀={params[0]*100:.1f}%, β₁={params[1]*100:.1f}%, β₂={params[2]*100:.1f}%, λ={params[3]:.1f}',
                 fontsize=12, fontweight='bold')
        plt.xlabel('Maturity (Years)')
        plt.ylabel('Yield (%)')
        plt.grid(True, alpha=0.3)
        plt.xlim(0, 30)
    
    plt.suptitle('Nelson-Siegel Model: Different Yield Curve Shapes', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

demonstrate_nelson_siegel_components()


## 🎛️ Chapter 3: Interactive Parameter Exploration

Now let's explore how changing each parameter affects the yield curve shape in real-time. This interactive section helps you develop intuition for how the Nelson-Siegel model works.

### Treasury Bonds (Nominal Yields)


In [ ]:
# Create interactive Treasury yield curve explorer
if HAS_WIDGETS:
    print("🎛️ Creating interactive Treasury yield curve explorer...")
    
    # Optional: Add your FRED API key here for real market data
    # fred_api_key = "your_fred_api_key_here"
    fred_api_key = None  # Will use synthetic data
    
    treasury_explorer = InteractiveYieldCurveExplorer('treasury', fred_api_key)
    treasury_dashboard = treasury_explorer.create_full_interactive_dashboard()
    
    display(treasury_dashboard)
    
else:
    print("📊 Interactive exploration not available. Install ipywidgets for full functionality.")
    
    # Show static example instead
    analyzer = YieldCurveAnalyzer()
    plotter = YieldCurvePlotter()
    
    # Example Treasury analysis
    example_yields = {1: 0.025, 2: 0.028, 5: 0.032, 10: 0.035, 30: 0.038}
    result = analyzer.analyze_single_curve('treasury', yields_data=example_yields)
    fig = plotter.plot_yield_curve_fit(result, show_deviations=True)
    plt.show()
